In [1]:
import numpy as np
import argparse
import cv2 as cv
import subprocess
import time
import os
from yolo_utils import infer_image, show_image

In [11]:
  FLAGS = []

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-m', '--model',
        default='./weights/')
    
    parser.add_argument('-w', '--weights',
        default='./weights/yolov3.weights')

    parser.add_argument('-cfg', '--config',
        default='./cfg/yolov3.cfg')

    parser.add_argument('-v', '--video-path',
        default='./videos/airport.mp4')

    parser.add_argument('-vo', '--video-output-path',
        default='./output/airport.avi')

    parser.add_argument('-l', '--labels',
        default='./data/coco.names')

    parser.add_argument('-c', '--confidence',
        type=float,
        default=0.5)

    parser.add_argument('-th', '--threshold',
        type=float,
        default=0.3)

    parser.add_argument('-t', '--show-time',
        default=False)

    FLAGS, unparsed = parser.parse_known_args()


# Get the labels
    labels = open(FLAGS.labels).read().strip().split('\n')

# Intializing colors to represent each label uniquely
    colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

# Load the weights and configutation to form the pretrained YOLOv3 model
    net = cv.dnn.readNetFromDarknet(FLAGS.config, FLAGS.weights)

    # Get the output layer names of the model
    layer_names = net.getLayerNames()
    layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [ ]:
 if FLAGS.video_path:
# Read the video
        try:
            vid = cv.VideoCapture(FLAGS.video_path)
            height, width = None, None
            writer = None
        except:
            raise 'Video cannot be loaded!\n\
                               Please check the path provided!'

        finally:
            while True:
                grabbed, frame = vid.read()

# Checking if the complete video is read
                if not grabbed:
                    break

                if width is None or height is None:
                    height, width = frame.shape[:2]

                frame, _, _, _, _ = infer_image(net, layer_names, height, width, frame, colors, labels, FLAGS)

                if writer is None:
 # Initialize the video writer
                     fourcc = cv.VideoWriter_fourcc(*"MJPG")
                     writer = cv.VideoWriter(FLAGS.video_output_path, fourcc, 30, 
                               (frame.shape[1], frame.shape[0]), True)

                writer.write(frame)

            print ("[INFO] Cleaning up...")
            writer.release()
            vid.release()
